In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Viterbi Algorithm

\begin{equation}
 \Pi^*= argmax_{} P(X ,\Pi)
\end{equation}

In [2]:
def viterbi(x, transmat, emissions):
    v_seq = np.zeros((len(x), len(transmat)))
    states = np.zeros((len(x), len(transmat)))
    best_states = np.zeros(len(x),  dtype=int)
    for i in range(len(v_seq)):
        if i == 0:
            v_seq[0] = (1/len(transmat)) * emissions[:,x[i]]
            v_seq[0] = np.log(v_seq[0])
            states[0] = np.zeros(len(transmat))
            continue
        p = v_seq[i-1] + np.log(transmat.T)
        states[i] =  np.argmax(p, axis=1)
        v_seq[i] =   np.log(emissions[:,x[i]]) + np.amax(p, axis = 1)
        
    best_states[-1] = np.argmax(v_seq[-1])
    
    for i in reversed(range(1, len(x))):
        best_states[i-1] = states[i][best_states[i]]
        
    return v_seq, states, best_states
    

# Forward Algorithm

\begin{equation}
\label{eq:bayes}
f_{(k)}(i) =  P(X_{1}....X_{i} |\Pi_{i} = k)
\end{equation}

In [3]:
def forward(x, transmat, emissions):
    f_seq = np.zeros((len(x), len(transmat)))
    s_seq = np.zeros((len(x), 1))
    a = 10**len(f_seq)
    for i in range(len(f_seq)):
        if i == 0:
            f_seq[i] =  emissions[:,x[i]] * (1/len(transmat)) *a
            continue
        f_seq[i] = emissions[:,x[i]] * np.matmul(f_seq[i-1], transmat)
        
    P = (1/len(transmat))*sum(f_seq[-1]) 
        
    return (f_seq, P, a)

# Backward Algorithm

\begin{equation}
\label{eq:bayes}
B_{(k)}(i) =  P(X_{i+1}....X_{L} |\Pi_{i} = k)
\end{equation}

In [4]:
def backward(x, transmat, emissions):
    b_seq = np.zeros((len(x), len(transmat)))
    a = 10**len(b_seq)
    for i in reversed(range(len(b_seq))):
        if i == len(b_seq) -1:
            b_seq[i] =  (1/len(transmat))*np.ones(len(transmat))*a
            continue
        b_seq[i] = np.matmul(emissions[:,x[i+1]] * b_seq[i+1],  transmat.T)
    
    P = (1/len(transmat))*sum(emissions[:,x[0]] * b_seq[0])
    return (b_seq, P, a)

# Let's predict CpG islands

In [5]:
seq = 'AGAACTGCTGACTGATAGATCGCGCGCCGCGCGCGCGGCATGCATCGATGCATGCGCAACTCGAATCGATTGACAGAAGACAATTATGACAATGCGCAGCAGCATTTATTACTACACATACGGTGCACTGCGCGCGCTCACTGCA'

# Let's get the viterbi

# Let's plot posterior probabilities

\begin{equation}
\label{eq:bayes}
P(\pi(i) = k|\textbf{X}) =  \frac{P(X_{1}....X_{i} |\Pi_{i} = k) P(X_{i+1}....X_{L} |\Pi_{i} = k)}{P(\textbf{X})} 
\end{equation}

## Which is just:

\begin{equation}
\label{eq:bayes}
P(\pi(i) = k|\textbf{X}) =  \frac{f_{(k)}(i) B_{(k)}(i)}{P(\textbf{X})} 
\end{equation}

## Putting all together